# EPA's Greenhouse Gas Reduction Fund
## Estimating the Effects  of Justice40 Requirements on Low-Income and Disadvantaged Communities

The Greenhouse Gas Reduction Fund (GGRF) is a historic investment by the U.S. government in clean energy and climate change mitigation projects in communities across America. The GGRF was established as part of 2022's Inflation Reduction Act, which stipulated that 40%-100% of the funds made available through the GGRF must go toward low-income and disadvantaged communites, as part of President Biden's Justice40 initiative. This analysis aims to:

1. Estimate the amount of additional funding going towards disadvantaged communities as a result of Justice40 requirements, compared to alternative scenarios in which funds are distributed based on population or real estate values, 
2. Identify the communities where Justice40 requirements made the largest difference, and 
3. Identify communities where the largest gaps in investment need still exist, based on the Climate Policy Intitiative's estimates of the need for investments in decarbonization to reach economy-wide net zero emissions by 2050.

#### Hypotheses
1. The amount of GGRF funding flowing to Justice40 communities will be roughly 75% higher than if funds were allocated based on population, and more than double the amount if funds were based on real estate values
2. Public & private investment supported by the GGRF funds will be less than 15% of the investment needs in Justice40 communities by 2035, and gaps will be higher in some communitieses

In [1]:
import pandas as pd

#load nri_limited dataset
nri_lim = pd.read_csv("./nri_limited.csv")

In [2]:
#Load GGRF communities dataset (to idenfity low-income and disadvantaged communities)
ggrf = pd.read_csv("./GGRF_communities.csv")

/tmp/ipykernel_6688/44929829.py:2: DtypeWarning: Columns (18,26,70,72,85,131) have mixed types. Specify dtype option on import or set low_memory=False.
  ggrf = pd.read_csv("./GGRF_communities.csv")


In [3]:
#Merge NRI and GGRF data by census tract number
ggrf_merged = pd.merge(ggrf, nri_lim, left_on="Census tract 2010 ID", right_on="TRACTFIPS", how="outer")

In [4]:
ggrf_merged["county_clean"] = ggrf_merged["County Name"].str.replace(" County", "")

#Create column with combined state and county name to avoid summing the same county names in different states in later steps
ggrf_merged["state-county"] = ggrf_merged["county_clean"] + " " + ggrf_merged["State/Territory"]

In [5]:
#Define total GGRF funding and the portion dedicated in the law to communities identified as disadvantaged (55% weighted average across three funds)
total_ggrf = 27000000000
j40_funding = total_ggrf*.55

#Within disadvantaged communities, calculate population as a percentage of total
ggrf_j40 = ggrf_merged[ggrf_merged["Identified as disadvantaged"] == True]

j40_total_pop = ggrf_j40["Total population"].sum()
j40_total_value = ggrf_j40["BUILDVALUE"].sum()

ggrf_j40["j40_pop_percent"] = ggrf_j40["Total population"] / j40_total_pop
ggrf_j40["j40_value_percent"] = ggrf_j40["BUILDVALUE"] / j40_total_value

#Estimate the portion dedicated to Justice40 communities by population in each census tract
ggrf_j40["j40_pop_weighted_funding"] = j40_funding * ggrf_j40["j40_pop_percent"]

#Estimate the portion dedicated to Justice40 communities by building value in each census tract
ggrf_j40["j40_value_weighted_funding"] = j40_funding * ggrf_j40["j40_value_percent"]


/tmp/ipykernel_6688/1486992115.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ggrf_j40["j40_pop_percent"] = ggrf_j40["Total population"] / j40_total_pop
/tmp/ipykernel_6688/1486992115.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ggrf_j40["j40_value_percent"] = ggrf_j40["BUILDVALUE"] / j40_total_value
/tmp/ipykernel_6688/1486992115.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

# Scenario 1: Estimate GGRF funding by Census Tract if Justice40 Rules Weren't in Place

In [6]:
all_total_pop = ggrf_merged["Total population"].sum()
all_total_value = ggrf_merged["BUILDVALUE"].sum()

ggrf_merged["all_pop_percent"] = ggrf_merged["Total population"] / all_total_pop
ggrf_merged["all_value_percent"] = ggrf_merged["BUILDVALUE"] / all_total_value

#Merge ggrf_j40 and ggrf_merged datasets
ggrf_j40 = pd.merge(ggrf_merged, ggrf_j40, on="Census tract 2010 ID", how="inner")

ggrf_j40["all_pop_weighted_funding"] = ggrf_j40["all_pop_percent"] * total_ggrf
ggrf_j40["all_value_weighted_funding"] = ggrf_j40["all_value_percent"] * total_ggrf

ggrf_merged = pd.merge(ggrf_merged, ggrf_j40, on="Census tract 2010 ID", how="outer")

#Calculate difference between scenario 1 funding and J40-based funding etimate per census tract
ggrf_merged["s1_vs_j40_funding"] = ggrf_merged["j40_pop_weighted_funding"] - ggrf_merged["all_pop_weighted_funding"]

s1_sorted_diff = ggrf_merged[["Census tract 2010 ID", "state-county", "s1_vs_j40_funding"]].sort_values(by = "s1_vs_j40_funding", ascending = False)

s1_sorted_diff.head(20)

,Census tract 2010 ID,state-county,s1_vs_j40_funding
63559,4.814101e+10,El Paso Texas,2.232748e+06
9724,6.073019e+09,San Diego California,2.083103e+06
16972,1.209704e+10,Osceola Florida,1.177544e+06
16954,1.209704e+10,Osceola Florida,1.173674e+06
25060,1.809738e+10,Marion Indiana,1.142175e+06
20810,1.602702e+10,Canyon Idaho,1.081704e+06
4601,6.029003e+09,Kern California,1.009676e+06
62347,4.806101e+10,Cameron Texas,9.747377e+05
50076,3.718305e+10,Wake North Carolina,9.721038e+05
9113,6.071009e+09,San Bernardino California,9.712438e+05


In [7]:
s1_grouped_ggrf_merged = ggrf_merged.groupby("Identified as disadvantaged")["Total population"].sum().reset_index(name="population")
s1_grouped_ggrf_merged["population_percent"] =  s1_grouped_ggrf_merged["population"] / s1_grouped_ggrf_merged["population"].sum()
s1_grouped_ggrf_merged["s1_ggrf_funding"] =  s1_grouped_ggrf_merged["population_percent"] * total_ggrf

total_s1_diff = j40_funding - s1_grouped_ggrf_merged[s1_grouped_ggrf_merged["Identified as disadvantaged"] == True]["s1_ggrf_funding"]
total_s1_diff_b = total_s1_diff / 1000000000
total_s1_diff_b = round(total_s1_diff_b, 2)
total_s1_diff_b = str(total_s1_diff_b.iloc[0])
s1_message = "As a result of the law's Justice40 requirements, " + "$" + total_s1_diff_b + " billion in additional funding went toward disadvanted and low-income communites, compared to a scenario in which funding is distributed based on population alone."

s1_percentage_increase = ggrf_j40["j40_value_weighted_funding"].sum() /  s1_grouped_ggrf_merged[s1_grouped_ggrf_merged["Identified as disadvantaged"] == True]["s1_ggrf_funding"].sum()

s1_grouped_ggrf_merged

,Identified as disadvantaged,population,population_percent,s1_ggrf_funding
0,False,219078043.0,0.667376,1.801916e+10
1,True,109189666.0,0.332624,8.980844e+09


In [8]:
print(s1_message)

As a result of the law's Justice40 requirements, $5.87 billion in additional funding went toward disadvanted and low-income communites, compared to a scenario in which funding is distributed based on population alone.


In [9]:
print("Percentage increase in GGRF funding to disadvantaged communities as a reuslt of Justice40 requirements under this scenario:")
round((s1_percentage_increase-1)*100, 2)

Percentage increase in GGRF funding to disadvantaged communities as a reuslt of Justice40 requirements under this scenario:


65.35

In [10]:
#how much total buildvalue is in j40 communites vs nonj40? multiply this % by total ggrf funding to get total difference
#how much population is in j40 vs nonj40? multiply by total ggrf funding to get total difference

s2_grouped_ggrf_merged = ggrf_merged.groupby("Identified as disadvantaged")["BUILDVALUE"].sum().reset_index(name="building_value")
s2_grouped_ggrf_merged["building_value_percent"] =  s2_grouped_ggrf_merged["building_value"] / s2_grouped_ggrf_merged["building_value"].sum()
s2_grouped_ggrf_merged["s2_ggrf_funding"] =  s2_grouped_ggrf_merged["building_value_percent"] * total_ggrf

total_s2_diff = j40_funding - s2_grouped_ggrf_merged[s2_grouped_ggrf_merged["Identified as disadvantaged"] == True]["s2_ggrf_funding"]
total_s2_diff_b = total_s2_diff / 1000000000
total_s2_diff_b = round(total_s2_diff_b, 2)
total_s2_diff_b = str(total_s2_diff_b.iloc[0])
s2_message = "As a result of the law's Justice40 requirements, " + "$" + total_s2_diff_b + " billion in additional funding went toward disadvanted and low-income communites, compared to a scenario in which funding is distributed based on real estate values alone."

s2_percentage_increase = ggrf_j40["j40_value_weighted_funding"].sum() /  s2_grouped_ggrf_merged[s2_grouped_ggrf_merged["Identified as disadvantaged"] == True]["s2_ggrf_funding"].sum()

s2_grouped_ggrf_merged

,Identified as disadvantaged,building_value,building_value_percent,s2_ggrf_funding
0,False,3.275931e+13,0.701471,1.893972e+10
1,True,1.394155e+13,0.298529,8.060278e+09


In [11]:
print(s2_message)

As a result of the law's Justice40 requirements, $6.79 billion in additional funding went toward disadvanted and low-income communites, compared to a scenario in which funding is distributed based on real estate values alone.


In [12]:
print("Percentage increase in GGRF funding to disadvantaged communities as a reuslt of Justice40 requirements under this scenario:")
round((s2_percentage_increase-1)*100, 2)

Percentage increase in GGRF funding to disadvantaged communities as a reuslt of Justice40 requirements under this scenario:


84.24

# Scenario 2: Estimate GGRF funding by Census Tract if Justice40 Rules Weren't in Place
#### Weighted by building value

In [13]:
#Calculate difference between scenario 2 funding and J40-based funding etimate per census tract
ggrf_merged["s2_vs_j40_funding"] = ggrf_merged["j40_value_weighted_funding"] - ggrf_merged["all_value_weighted_funding"]

s2_sorted_diff = ggrf_merged[["Census tract 2010 ID", "state-county", "s2_vs_j40_funding"]].sort_values(by = "s2_vs_j40_funding", ascending = False)

s2_sorted_diff.head(20)

,Census tract 2010 ID,state-county,s2_vs_j40_funding
65484,4.834795e+10,Nacogdoches Texas,2.983119e+07
14047,1.100101e+10,District of Columbia District of Columbia,1.184056e+07
61397,4.715702e+10,Shelby Tennessee,7.073473e+06
42896,3.501300e+10,Doña Ana New Mexico,6.021224e+06
70148,5.303303e+10,King Washington,5.056453e+06
8962,6.071002e+09,San Bernardino California,4.137583e+06
27541,2.107192e+10,Floyd Kentucky,4.087052e+06
65479,4.834395e+10,Morris Texas,3.937483e+06
9724,6.073019e+09,San Diego California,3.801276e+06
41870,3.402301e+10,Middlesex New Jersey,3.713699e+06


In [14]:
#Sum J40-based population-weighted funding by state-county
county_j40_pop_weighted_funding = ggrf_merged.groupby("state-county")["j40_pop_weighted_funding"].sum().reset_index(name="pop_weighted_funding")
county_j40_pop_weighted_funding

#Sum J40-based building value-weighted funding by state-county
county_j40_value_weighted_funding = ggrf_merged.groupby("state-county")["j40_value_weighted_funding"].sum().reset_index(name="value_weighted_funding")

#load CPI investment needs estimate by county
cpi = pd.read_csv("./ggrf_needs.csv")

#Create cleaned state-county column
cpi["county_clean"] = cpi["County"].str.replace(" County", "")
cpi["state-county"] = cpi["county_clean"] + " " + cpi["State"]

#Adjust currency units from thousands of dollars to dollars
cpi["Investment ($'000)"] = cpi["Investment ($'000)"]*1000

#Create dataset filtered for investment needs in buildings only
cpi_buildings = cpi[cpi["GGRF category"] == "Buildings"]

#Sum CPI investment needs by county
cpi_county = cpi.groupby("state-county")["Investment ($'000)"].sum().reset_index(name="total_investment_need")
cpi_buildings_county = cpi_buildings.groupby("state-county")["Investment ($'000)"].sum().reset_index(name="total_investment_need_buildings")

#Merge the summarized CPI investment needs dataset with Justice40-based funding estimates by population and by building values
cpi_merged_pop = pd.merge(cpi_county, county_j40_pop_weighted_funding, on="state-county")
cpi_merged_value = pd.merge(cpi_buildings_county, county_j40_value_weighted_funding, on="state-county")

#Compare the CPI investment needs per county to estimated J40-based funding in each dataset
cpi_merged_pop["total_unmet_need"] = cpi_merged_pop["total_investment_need"] - cpi_merged_pop["pop_weighted_funding"]
cpi_merged_value["total_unmet_need_buildings"] = cpi_merged_value["total_investment_need_buildings"] - cpi_merged_value["value_weighted_funding"]

s1_unmet_need = cpi_merged_pop["total_unmet_need"].sum()
s1_unmet_need_t = s1_unmet_need / 1000000000000
s1_unmet_need_t = round(s1_unmet_need_t, 2)
s1_unmet_need_t = str(s1_unmet_need_t)

percentage_met = ((cpi_merged_pop["pop_weighted_funding"].sum()*7) / cpi_merged_pop["total_investment_need"].sum())*100

cpi_merged_pop.sort_values(by="total_unmet_need", ascending=False)

,state-county,total_investment_need,pop_weighted_funding,total_unmet_need
1692,Los Angeles California,1.628127e+11,6.733184e+08,1.621394e+11
1200,Harris Texas,7.540858e+10,2.901623e+08,7.511842e+10
623,Cook Illinois,7.117617e+10,2.981901e+08,7.087798e+10
1763,Maricopa Arizona,6.754349e+10,1.603303e+08,6.738316e+10
804,Douglas Nebraska,6.469283e+10,2.070479e+07,6.467212e+10
...,...,...,...,...
2590,Slope North Dakota,1.631766e+07,0.000000e+00,1.631766e+07
2212,Petroleum Montana,1.184349e+07,0.000000e+00,1.184349e+07
1465,Kenedy Texas,8.191048e+06,7.724907e+04,8.113799e+06
1485,King Texas,7.426862e+06,3.223245e+04,7.394630e+06


In [15]:
print(s1_unmet_need_t + " trillion in unmet national gross investment needs to 2035, based on population-weighted estimates")

6.57 trillion in unmet national gross investment needs to 2035, based on population-weighted estimates


In [16]:
print("Percentage of decarbonization investment need catalyzed by GGRF grants:")
round(percentage_met, 2)

Percentage of decarbonization investment need catalyzed by GGRF grants:


1.53

In [17]:
s2_unmet_need = cpi_merged_value["total_unmet_need_buildings"].sum()
s2_unmet_need_t = s2_unmet_need / 1000000000000
s2_unmet_need_t = round(s2_unmet_need_t, 2)
s2_unmet_need_t = str(s2_unmet_need_t)

percentage_met_buildings = ((cpi_merged_value["value_weighted_funding"].sum()*7) / cpi_merged_value["total_investment_need_buildings"].sum())*100

cpi_merged_value.sort_values(by="total_unmet_need_buildings", ascending=False)

,state-county,total_investment_need_buildings,value_weighted_funding,total_unmet_need_buildings
1692,Los Angeles California,2.664020e+10,4.906435e+08,2.614955e+10
1200,Harris Texas,1.206556e+10,1.657786e+08,1.189978e+10
623,Cook Illinois,1.171645e+10,3.166565e+08,1.139980e+10
1763,Maricopa Arizona,1.067058e+10,1.512736e+08,1.051931e+10
2472,San Diego California,7.965992e+09,8.601862e+07,7.879973e+09
...,...,...,...,...
210,Blaine Nebraska,7.944526e+05,0.000000e+00,7.944526e+05
84,Arthur Nebraska,6.976376e+05,1.705523e+05,5.270853e+05
1485,King Texas,5.713369e+05,1.137485e+05,4.575884e+05
1465,Kenedy Texas,4.999198e+05,1.431692e+05,3.567506e+05


In [18]:
print(s2_unmet_need_t + " trillion in unmet national gross investment needs in building decarbonization to 2035.")

0.79 trillion in unmet national gross investment needs in building decarbonization to 2035.


In [19]:
print("Percentage of building decarbonization investment need catalyzed by GGRF grants:")
round(percentage_met_buildings, 2)

Percentage of building decarbonization investment need catalyzed by GGRF grants:


12.48

# Conclusions

Due to the Justice40 requirements in the law, $5.87 billion in additional funding will be directed toward disadvantaged communities compared to an alternative scenario in which the funds were distributed based on population alone, representing an increase in funding of 65%. This is somewhat lower than the amount hypothesized, but still significant.

Likewise, 84% more funding, or $6.79 billion will go to disadvantaged communities compared to a scenario in which funding were distributed based on real estate values alone, nearly matching the hypothesis that funding would be doubled in comparison.

The investment needs to 2025 are far greater than the amount of investment that will be catalyzed by the GGRF. If GGRF grants are leveraged by approximately 7 times, in line with EPA estimates, they will only meet 1.53% of the need for gross investment in decarbonization to 2035 nationally. A large portion of GGRF grants and private investment catalyzed by the GGRF will go toward building decarbonization, but even assuming that 100% of this investment is dedicated to buildings, that represents 12.47% of the need for investment in building decarbonization to 3025.
